# LLM generation text detection

In [1]:
!pip install transformers
!pip install tensorflow_text


In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
train_daigt = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv")

train_essays.head()
#train_prompts.head()

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


## Datasets aligment

In [7]:
train_daigt["generated"] = train_daigt["label"]
train_essays = train_essays.merge(train_prompts, on='prompt_id', how='inner')

train_essays = pd.concat([train_essays,train_daigt])

train_essays.loc[:,["prompt_name","generated"]].value_counts()


train_essays["text"] = train_essays["text"].map(lambda x : x.lower())


## BERT

In [13]:
import keras
import keras_nlp
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split

X = train_essays["text"].values
y = train_essays["generated"].values


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Assuming your labels are 0 and 1
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


preprocessor = keras_nlp.models.BertPreprocessor.from_preset(
    "bert_large_en_uncased",
)

# Pretrained classifier without preprocessing.
classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_large_en_uncased",
    num_classes = 2,
    preprocessor = preprocessor,
    
)
# Access backbone programmatically (e.g., to change `trainable`).
classifier.backbone.trainable = False

classifier.compile(
    loss = keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(5e-5),
    jit_compile = True,
)

history = classifier.fit(
    x = X_train,
    y = y_train,
    validation_data = (X_val, y_val),
    batch_size = 64,
    epochs = 10,  # Set the number of epochs as required
    class_weight = class_weights
)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend
1341009960/1341009960 [==============================] - 51s 0us/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
## Evaluation and submission

In [ ]:
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
submission_dict = {"id":[],"generated":[]}

for _,row in test_essays.iterrows():
    submission_dict["id"].append(row["id"])
    submission_dict["generated"].append(classifier.predict(row["text"])[1])

submission = pd.DataFrame.from_dict(submission_dict)
submission.to_csv("/kaggle/working/submission.csv")

    
